# Structured Output
<img src="./assets/LC_StructuredOutput.png" width="500">


## Setup

Load and/or check for needed environmental variables

In [0]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

## Structured Output Example

In [0]:
from typing_extensions import TypedDict

from langchain.agents import create_agent


class ContactInfo(TypedDict):
    name: str
    email: str
    phone: str


agent = create_agent(model="openai:gpt-5-mini", response_format=ContactInfo)

recorded_conversation = """We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

#### Multiple data types are supported 

* pydantic `BaseModel`
* `TypedDict`
* `dataclasses`
* json schema (dict)

In [0]:
from langchain.agents import create_agent
from pydantic import BaseModel


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(model="openai:gpt-5-mini", response_format=ContactInfo)

recorded_conversation = """ We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]